In [21]:
# https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb

In [33]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [5]:
# Загрузка доукментов
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()
# Разбиение доукментов
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [ ]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

In [ ]:
docs[0]

In [ ]:
model_path = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    device_map='auto',
)

In [17]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [20]:
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)

In [31]:
def test_model(tokenizer, pipeline, prompt_to_test):

    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        truncation=True,
        max_length=200)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [32]:
test_model(tokenizer,
           query_pipeline,
           "Please explain what is the State of the Union address. Give just a definition. Keep it in 20 words.")

Test inference: 3.665 sec.
Result: Please explain what is the State of the Union address. Give just a definition. Keep it in 20 words.
What's the difference between the State of the Union, the State of the Union Address, the National Address (the State of the Union Message), the State of Address, and the State of the Union (the National Address)?
How many times is the President's State of the Union address?
The President speaks once a year.
Where do we find the State of the Union addresses?
There is a State of the Union Address each year at the beginning (or beginning of each) new congress (Congresses in the U.S. are divided into two chambers, the U.S. Senate and the U.S. House of Representatives).
Why do they call it the State of the Union address?
The State of the Union Address is given every year at the beginning of a new congress. The first


In [35]:
from langchain_community.embeddings import GPT4AllEmbeddings

In [36]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings(), persist_directory="chroma_db")

In [37]:
question = "What are the approaches to Task Decomposition?"
docs = vectordb.similarity_search(question)
len(docs)

4

In [38]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

In [39]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [40]:
question = "What are the approaches to Task Decomposition?"

In [43]:
# qa.invoke(question)